In [1]:
### Run environment setup
import os
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression
import pandas as pd
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

### Import general utilities for notebook
import lib.BBData
from lib.BBDataLoad import load_char_df

### Define folders
base_folder = os.getcwd()

2023-03-08 09:47:01.678867: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 09:47:01.931204: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-08 09:47:01.931256: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-08 09:47:03.119455: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
def sentence_preprocess(sentence, stopwords=stopwords.words('english'), min_sentence_len=3):
    """
    This function can be used in order to preprocess a given character sentence
    ## Params
    * `sentence`: the sentence to preprocess
    * `stopwords`: is a set of stopwords, the default value of such parameter is the set of 
    english stopwords of the library nltk
    * `min_sentence_len`: is the minimum lenght a sentence should have in order to be preprocessed
    and remain in the dataset
    ## Returns
    The preprocessed sentence
    """
    # lower sentence
    sentence = sentence.lower()
    # substitute punctuations with single space
    sentence = re.sub(r'[^A-Za-z\s]', ' ', sentence)
    # split sentences
    sentence_splitted = sentence.split()
    # check if the sentence is too short
    is_short = len(sentence_splitted) <= min_sentence_len
    # check if the sentence contains at least a non stopword
    is_relevant = False
    for word in sentence_splitted:
        if word not in stopwords:
            is_relevant = True
        else:
            # remove stopwords
            sentence = sentence.replace(word, "")
    # substitutes multiple spaces with one single
    sentence = re.sub(r'\s+', ' ', sentence)
    # remove left and right space 
    sentence = sentence.lstrip().rstrip()
    # returns the sentence preprocessed and the final relence of it
    return sentence, (not is_short and is_relevant)

In [18]:
df = load_char_df('Barney', base_folder)
docs =  df['train']['response'] + df['test']['response'] + df['val']['response']
print(len(docs))
docs_prepr = docs
# docs_prepr = []
# y = []
# for d in docs:
#     snt_prp = sentence_preprocess(d)
#     if snt_prp[1]:
#         docs_prepr.append(snt_prp[0])
#         y.append(0)
# docs_prepr[:10]

Found cached dataset csv (/app/notebooks/cache/csv/default-dd07b0e7e3df8359/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at /app/notebooks/cache/csv/default-dd07b0e7e3df8359/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-c7d3ec6662680d64.arrow and /app/notebooks/cache/csv/default-dd07b0e7e3df8359/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-1dc91bae0d09a1f5.arrow
Loading cached split indices for dataset at /app/notebooks/cache/csv/default-dd07b0e7e3df8359/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-c3793660279940c7.arrow and /app/notebooks/cache/csv/default-dd07b0e7e3df8359/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-f96aafc4d74714b6.arrow


5194


In [19]:
topic_model= BERTopic(
        verbose=True
)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/163 [00:00<?, ?it/s]

2023-03-08 10:45:28,050 - BERTopic - Transformed documents to Embeddings
2023-03-08 10:45:37,565 - BERTopic - Reduced dimensionality
2023-03-08 10:45:37,932 - BERTopic - Clustered reduced embeddings


In [20]:
topic_model.visualize_topics()

In [21]:
topic_model.visualize_documents(docs)

Ora per Sheldon

In [83]:
df = pd.read_csv(os.path.join(base_folder, 'Sheldon.csv'))
docs = df['response'].to_list()
print(len(docs))
for d in docs:
    snt_prp = sentence_preprocess(d)
    if snt_prp[1]:
        docs_prepr.append(snt_prp[0])
        y.append(1)

11642


In [71]:
empty_dimensionality_model = BaseDimensionalityReduction()
clf = LogisticRegression()
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

In [73]:
docs_prepr[:5]

['hey k lwys thng hlf sn grls well got new fvorte lebnese grls lebnese grls re new hlf sns',
 'okay et bar fifteen mutes suit',
 'ut jut y ut wh put ut',
 'blzer',
 'see ths s forgotten sad nght met']

In [74]:
topic_model= BERTopic(
        umap_model=empty_dimensionality_model,
        hdbscan_model=clf,
        ctfidf_model=ctfidf_model,
        verbose=True
)
topics, probs = topic_model.fit_transform(docs_prepr, y=y)

Batches:   0%|          | 0/442 [00:00<?, ?it/s]

2023-03-05 17:48:02,228 - BERTopic - Transformed documents to Embeddings
2023-03-05 17:48:02,230 - BERTopic - The dimensionality reduction algorithm did not contain the `y` parameter and therefore the `y` parameter was not used
2023-03-05 17:48:02,232 - BERTopic - Reduced dimensionality
2023-03-05 17:48:02,969 - BERTopic - Clustered reduced embeddings


In [75]:
max(topics)

1

In [76]:
# topic_model.get_document_info(docs_prepr)

In [77]:
# topic_model.get_topic(0)

In [78]:
topic_model.visualize_documents(docs_prepr)

In [81]:
mappings = topic_model.topic_mapper_.get_mappings()
mappings = {value: y[key] for key, value in mappings.items()}

df = topic_model.get_topic_info()
df["Class"] = df.Topic.map(mappings)
df

,Topic,Count,Name,Class
0,0,9931,0_oh_im_well_dont,0
1,1,4202,1_ted_he_know_go,0


In [95]:
mappings = topic_model.topic_mapper_.get_mappings()
y_mapped = [mappings[val] for val in y]
y_mapped[-1]

0

In [98]:
y[-10:]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]